In [1]:
import geopandas as gpd
import pandas as pd

## Open State data

In [2]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Mobile Home Data Percents

In [3]:
acs_homeownership = pd.read_csv(
    "data/ACSDP5Y2023.DP04-2025-02-05T232725.csv", thousands=","
)

In [4]:
rename_columns_home_percent = {"Label (Grouping)": "NAME"}
for column in list(acs_homeownership.columns):
    str_split = column.split("!!")
    if len(str_split) == 2:
        if str_split[1] == "Percent":
            rename_columns_home_percent[column] = str_split[0].strip()
    else:
        continue

In [5]:
home_percents_df = (
    acs_homeownership[rename_columns_home_percent.keys()]
    .rename(columns=rename_columns_home_percent)
    .T.reset_index()
)
home_percents_df.columns = [column.strip() for column in list(home_percents_df.iloc[0])]
home_percents_df = home_percents_df[1:]

In [6]:
home_percents_df["MOBILE_HOME_PERCENT"] = (
    home_percents_df["Mobile home"].str.replace("%", "").astype(float)
)
home_percents_df = home_percents_df[["NAME", "MOBILE_HOME_PERCENT"]]

## Get Mobile Home Data Totals

In [9]:
rename_columns_home_totals = {"Label (Grouping)": "NAME"}
for column in list(acs_homeownership.columns):
    str_split = column.split("!!")
    if len(str_split) == 2:
        if str_split[1] == "Estimate":
            rename_columns_home_totals[column] = str_split[0].strip()
    else:
        continue

In [10]:
home_total_df = (
    acs_homeownership[rename_columns_home_totals.keys()]
    .rename(columns=rename_columns_home_totals)
    .T.reset_index()
)
home_total_df.columns = [column.strip() for column in list(home_total_df.iloc[0])]
home_total_df = home_total_df[1:]

In [11]:
home_total_df["MOBILE_HOME_TOTAL"] = home_total_df["Mobile home"].astype(int)
home_total_df = home_total_df[["NAME", "MOBILE_HOME_TOTAL"]]

In [12]:
home_df = home_percents_df.merge(home_total_df, how="inner", on="NAME")

## Merge Data

In [13]:
home_columns_percents_gdf = states_df.merge(home_df, on="NAME", how="inner").dropna()

In [14]:
home_columns_percents_gdf = home_columns_percents_gdf[
    ["NAME", "geometry", "MOBILE_HOME_PERCENT", "MOBILE_HOME_TOTAL"]
]

In [15]:
home_columns_percents_gdf = home_columns_percents_gdf.to_crs(9311)
home_columns_percents_gdf.to_file("data/mobile_home.gpkg")